In [1]:

#Imort modules
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img
from skimage.transform import resize
import pandas as pd
import s2sphere

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load the Tensorflow Hub modul
module = hub.Module("https://www.kaggle.com/models/google/planet-v2/frameworks/TensorFlow1/variations/planet-vision-classifier-planet-v2/versions/1")
height, width = hub.get_expected_image_size(module)

# Labelmap einladen
labelmap = pd.read_csv("planet_v2_labelmap.csv")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [2]:
# Funktion zur Bestimmung von x Maxima
def find_x_maxima(array, x):
    indices = np.argsort(array)[-x:][::-1]  # Indizes der x größten Elemente
    maxima = array[indices]  # Werte der x größten Elemente

    return indices, maxima

# Schreibt Ergebnisse fur ein Bild in txt Datei um diese in Matlab zu plotten
def text_datei(idx_nummer,score,dateiname):
    with open("Ergebnisse/Ergebnis-"+dateiname+".txt",'w') as datei:
        for i in range(5):
            id_nummer=idx_nummer[i]
            # Zellennummer im Labelmap finden
            corresponding_row = labelmap[labelmap['id'] == id_nummer]
            s2_cell_id_hex = corresponding_row['S2CellId'].values[0]

            # Konvertieren Sie die Hexadezimal-Zeichenfolge in einen S2-Zellen-ID-Objekt
            s2_cell_id = s2sphere.CellId.from_token(s2_cell_id_hex)

            # Erstellen Sie eine S2-Zelle aus der S2-Zellen-ID
            s2_cell = s2sphere.Cell(s2_cell_id)

            # Extrahieren Sie die Ecken der Zelle
            vertices = [s2sphere.LatLng.from_point(s2_cell.get_vertex(i)) for i in range(4)]

            # Konvertieren Sie die Koordinaten in Arrays
            latitudes, longitudes = zip(*[(vertex.lat().degrees, vertex.lng().degrees) for vertex in vertices])
            
            latitudes_array = np.array(latitudes)
            longitudes_array = np.array(longitudes)

            datei.write(str(latitudes_array[:]) + '\n')
            datei.write(str(longitudes_array[:]) + '\n')
            datei.write(str(score[i]) + '\n')
            datei.write(str(id_nummer)+ '\n')
    
# Wertet einzelne Bilder aus, um daraus Plots zu erzeugen
def bildanalyse(imagenamen,cellnr):
    images = np.zeros([len(imagenamen),299,299,3])
    for i in range(len(imagenamen)):
        # Bild einladen und passende Groesse erstellen
        image = img.imread("images/"+ imagenamen[i])
        #image = img.imread(imagenamen[i])
        image = image[:,:,0:3]
        image_res = resize(image, (299, 299)) # Bilder brauchen die Groesse 299x299 mit 3 Kanaelen zwischen 0 und 1
        images[i,:,:,:] = image_res[:,:,:] # Network kann mehrere Bilder gleichzeitig nehmen -> shape [n,299,299,3]

    # Neuronales Netz anwenden
    features = module(images)

    print('Netzwerk fertig')

    # in Numpy Array umwandeln
    with tf.compat.v1.Session() as sess:
        # Führen Sie die Sitzung aus und erhalten Sie den Wert des Tensors
        features_np = sess.run(features)

    print('NP-Array erzeugt')

    if cellnr==0:
        for i in range(len(imagenamen)):
            maxima_id,maxima_score = find_x_maxima(features_np[i,:],5)
            text_datei(maxima_id,maxima_score,imagenamen[i])
        print('Texdateien abgespeichert')
    else:
        print(features_np[:,cellnr])
    

    return features_np

# Wertet ein Bild mit verschiedenen scharzen Kaesten fuer Heatmap aus
def bildanalyse_heatmap(imagename,nmax,endung):
    images = np.zeros([nmax+1,299,299,3])

    # Original Bild 
    #image = img.imread("images/"+ imagename+".png")
    #image = img.imread("images/Segmented_imgs_"+imagename+"/Segmented.png")
    image = img.imread("images/output_"+imagename+"/"+imagename+endung)
    image = image[:,:,0:3]
    image_res = resize(image, (299, 299)) # Bilder brauchen die Groesse 299x299 mit 3 Kanaelen zwischen 0 und 1
    images[0,:,:,:] = image_res[:,:,:] # Network kann mehrere Bilder gleichzeitig nehmen -> shape [n,299,299,3]


    for i in range(nmax):
        # Bild einladen und passende Groesse erstellen
        #image = img.imread("images/"+ imagename + '_output_' + str(i) + '.png')
        #image = img.imread("images/"+ imagename + '_' + str(i+1) + '.png')
        #image = img.imread("images/Segmented_imgs_"+imagename+"/Segmented_"+str(i+1)+".png")
        image = img.imread("images/output_"+imagename+"/"+imagename+"_resize_output_"+str(i)+".jpg")
        #image = img.imread("images/output_"+imagename+"/"+imagename+"_output_"+str(i)+endung)
        image = image[:,:,0:3]
        image_res = resize(image, (299, 299)) # Bilder brauchen die Groesse 299x299 mit 3 Kanaelen zwischen 0 und 1
        images[i+1,:,:,:] = image_res[:,:,:] # Network kann mehrere Bilder gleichzeitig nehmen -> shape [n,299,299,3]

    plt.imshow(images[-1,:,:,:])
    plt.show

    print('Bilder eingelesen')
    
    # Neuronales Netz anwenden
    features = module(images)

    print('Netzwerk fertig')

    # in Numpy Array umwandeln
    with tf.compat.v1.Session() as sess:
        # Führen Sie die Sitzung aus und erhalten Sie den Wert des Tensors
        features_np = sess.run(features)

    print('NP-Array erzeugt')

    cell_id = np.argsort(features_np[0,:])[-1:][::-1]
    cell_prop = features_np[1:,cell_id]

    with open("Ergebnisse/Heatmap-"+imagename+".txt",'w') as datei:
        datei.write(str(cell_id)+'\n')
        datei.write(str(cell_prop[:,0])+'\n')

    print('Texdateien abgespeichert')

    return features_np, cell_prop
    



In [3]:
#names = ["Kennedybruecke","Kirschbluete","london","Muenster","Nussallee","Rathaus","Universitaet","Statue","Winter","Sommer"]
#n = [14,11,11,13,12,16,13,13,15,15,15]

#names = ["B_Kennedybruecke","B_Kirschbluete","B_Muenster","B_Nussallee","B_Rathaus","B_Universitaet","B_Statue","london","See_Sommer","See_Winter"]
#endungen = [".jpg",".jpg",".jpg",".png",".jpg",".jpg",".jpg",".png",".jpg",".jpg"]
#n = [96,96,96,96,96,96,96,100,96,96]

#names = ["Loewe","Matterhorn","Zoo"]
#n = [16,11,11]
names = ["Loewe_Krueger","Matterhorn","Loewe_Zoo"]
n = [96,96,96]

for i in range(0,len(names)):
    print(('Bild '+ names[i]+' wird prozessiert: '+str(i)+' von '+str(len(names))))
    features_np, cell_prop = bildanalyse_heatmap(names[i],n[i],".png")

Bild Loewe_Krueger wird prozessiert: 0 von 3
Bilder eingelesen
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Netzwerk fertig
NP-Array erzeugt
Texdateien abgespeichert
[0.12720557 0.1333016  0.11340791 0.10565481 0.09901606 0.08906078
 0.10108333 0.09069698 0.10753006 0.09622489 0.09400484 0.09783579
 0.09632114 0.08261222 0.09380652 0.08881146 0.1027843  0.07321994
 0.07574269 0.10277586 0.10464382 0.08782063 0.08794332 0.09397079
 0.09994581 0.09146746 0.08687489 0.10389145 0.09524098 0.07425065
 0.07933933 0.09154207 0.0845759  0.10452414 0.11505503 0.10614055
 0.09894593 0.07206841 0.06376991 0.09667823 0.08967791 0.09557208
 0.11368935 0.10484755 0.05751393 0.0753784  0.07062591 0.09473878
 0.08158936 0.09432796 0.09086132 0.12285332 0.06224329 0.10643497
 0.07950412 0.09540687 0.06948511 0.08676369 0.04814725 0.06232323
 0.05287928 0.09086021 0.10736245 0.09726776 0.08079905 0.09202757
 0.07329777 0.07247356 0.05506494 0.0971746  0.10877846 0.08887904
 0.09871556 0.08529006 0.08167348 0.07970084 0.09265506 0.09583975
 0.09272781 0.08427945 0.10381697 0.08894088 0.09963899 0.07731722
 0.1

In [4]:
#images=["london.png","See_Sommer.jpg","See_Winter.jpg","B_Kennedybruecke.jpg", "B_Kirschbluete.jpg","B_Muenster.jpg","B_Nussallee.png","B_Rathaus.jpg","B_Statue.jpg","B_Universitaet.jpg"]
images=["Loewe_Krueger.png","Loewe_Zoo.png","Matterhorn.png"]
features_ein = bildanalyse(images,0)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Netzwerk fertig
NP-Array erzeugt
Texdateien abgespeichert
